In [103]:
!pip install keras-tuner


In [104]:
import tensorflow as tf 
from tensorflow import keras
import numpy as np


In [13]:
fashion_mnist=keras.datasets.fashion_mnist

In [105]:
(train_images,train_labels),(test_images,test_labels)=fashion_mnist.load_data()

In [15]:
train_images=train_images/255.0
test_images=test_images/255.0

In [16]:
train_images[0].shape

(28, 28)

In [107]:
train_images=train_images.reshape(len(train_images),28,28,1)
test_images=test_images.reshape(len(test_images),28,28,1)

In [139]:
def build_model1(hp):
  model=keras.Sequential([
                          keras.layers.Conv2D(
                              filters=hp.Int('conv_1_filter',min_value=32,max_value=64,step=16),
                              kernel_size=hp.Choice('conv_1_kernel',values=[3,5]),
                              activation='relu',
                              input_shape=(28,28,1)
                          ),
                          keras.layers.Conv2D(
                              filters=hp.Int('conv_2_filter',min_value=32,max_value=64,step=16),
                              kernel_size=hp.Choice('conv_2_kernel',values=[3,5]),
                              activation='relu'
                          ),
                          keras.layers.Flatten(),
                          keras.layers.Dense(
                              units=hp.Int('dense_1_units',min_value=32,max_value=128,step=16),
                              activation='relu'
                          ),
                          keras.layers.Dense(10,activation='softmax')


                              
  ])
  model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate',values=[1e-2,1e-3])),
                 loss='sparse_categorical_crossentropy',
                 metrics=['accuracy'])
  return model


In [140]:
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

In [137]:
pip install -U keras-tuner

Requirement already up-to-date: keras-tuner in /usr/local/lib/python3.7/dist-packages (1.0.2)


In [147]:
tuner_search=RandomSearch(build_model1,
                          objective='val_accuracy',
                          max_trials=5,directory='output',project_name="Mnist_Fashion")

INFO:tensorflow:Reloading Oracle from existing project output/Mnist_Fashion/oracle.json
INFO:tensorflow:Reloading Tuner from output/Mnist_Fashion/tuner0.json


 tuner_search.search(train_images,train_labels,epochs=3,validation_split=0.1)

In [148]:
 tuner_search.search(train_images,train_labels,epochs=3,validation_split=0.1)
 

INFO:tensorflow:Oracle triggered exit


tuner_search.search(train_images,train_labels,epochs=3,validation_split=0.1)

In [144]:
model=tuner_search.get_best_models(num_models=1)[0]

In [145]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 24, 24, 48)        1248      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 22, 22, 48)        20784     
_________________________________________________________________
flatten (Flatten)            (None, 23232)             0         
_________________________________________________________________
dense (Dense)                (None, 112)               2602096   
_________________________________________________________________
dense_1 (Dense)              (None, 10)                1130      
Total params: 2,625,258
Trainable params: 2,625,258
Non-trainable params: 0
_________________________________________________________________
